An simple text classifier using Semantic Kernel and OpenAI.

This example is derived from this article: https://systenics.ai/blog/2024-07-26-building-a-simple-nlp-classifier-with-semantic-kernel/

In [ ]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.17.2"

using Microsoft.Extensions.Configuration;
using System.IO;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

public static var config = new ConfigurationBuilder()
          .AddJsonFile(Path.GetFullPath("secrets.json"), optional: false, reloadOnChange: true)
          .Build();

public static string OpenAIKey =  config["openai-key"];

Create enum and add prompts to dictionary

In [5]:
public enum PromptType
 {
     Simple,
     Specific,
     Structured,
     FewShot,
     AvoidGuessing
 }
var prompts = new Dictionary<PromptType, string>
{
{ PromptType.Simple, "Classify the following text: {{$request}}" },
{ PromptType.Specific, "Classify the following text as one of the following: Historical, Anecdote, Folklore, Fiction, Unknown.\nText: {{$request}}" },
{ PromptType.Structured, "Instructions: Classify the following text as one of the following types:\nChoices: Historical, Anecdote, Folklore, Fiction, Unknown.\nUser Input: {{$request}}\nType: " },
{ PromptType.FewShot, @"Instructions: Classify the following text as one of the following types:
Choices: Historical, Anecdote, Folklore, Fiction, Unknown.

User Input: The American Revolution began in 1775.
Type: Historical

User Input: I once saw a shooting star while camping.
Type: Anecdote

User Input: The legend of Robin Hood stealing from the rich to give to the poor.
Type: Folklore

User Input: Harry Potter is a young wizard who attends Hogwarts School of Witchcraft and Wizardry.
Type: Fiction

User Input: The quantum properties of dark matter.
Type: Unknown

User Input: {{$request}}
Type: " },
{ PromptType.AvoidGuessing, @"Instructions: Classify the following text. If you don't know the type or it doesn't fit the given categories, respond with ""Unknown"".
Choices: Historical, Anecdote, Folklore, Fiction, Unknown.

User Input: {{$request}}
Type: " }
};

Set up the kernel

In [6]:
static string ModelId = "gpt-3.5-turbo-0125";

// Create a kernel with OpenAI chat completion
var builder = Kernel.CreateBuilder()
.AddOpenAIChatCompletion(ModelId, OpenAIKey);

Kernel kernel = builder.Build();

Make the AI call to classify an example

In [7]:
string input = "The answer to the Ultimate Question of Life, the Universe, and Everything is 42";

//Pass in the input as kernel arguments 
KernelArguments args = new KernelArguments() { { "request", input } };

//Select a template 
PromptType selectedPrompt = PromptType.Specific;
string promptTemplate = prompts[selectedPrompt];

//Make the AI call and get the response 
FunctionResult result = await kernel.InvokePromptAsync(promptTemplate, args);

string aiResponse = result.GetValue<string>();

aiResponse

Fiction